<h1>游戏按键绑定

In [1]:
# 判断居中人体框
def get_center(box):
    x1, y1, x2, y2 = box
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return center_x, center_y

def find_closest_box(boxes, screen_width, screen_height):
    screen_center = (screen_width / 2, screen_height / 2)
    min_distance = float('inf')
    closest_box = None

    for box in boxes:
        center = get_center(box)
        distance = np.sqrt((center[0] - screen_center[0]) ** 2 + (center[1] - screen_center[1]) ** 2)
        if distance < min_distance:
            min_distance = distance
            closest_box = box

    return closest_box
def find_box_index(boxes, target_box):
    for index, box in enumerate(boxes):
        if np.all(box == target_box):
            return index
    return -1  # 如果找不到目标box，返回-1

<h1>将模型预测值转化为键盘输入信号

In [2]:
# 导入BaseNN
from BaseNN import nn
from BaseML import Classification as cls
from XEdu.hub import Workflow as wf
import numpy as np
import pandas as pd
import json
import requests
import time

In [3]:
from PIL import Image, ImageDraw, ImageFont
def draw_box_string(img, x, y, string, size=30, color=(255, 255, 255)):
    """
    img: imread读取的图片;
    x,y:字符起始绘制的位置;
    string: 显示的文字;
    return: img
    """
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    draw = ImageDraw.Draw(img)
    # simhei.ttf 是字体，你如果没有字体，需要下载
    font = ImageFont.truetype("simhei.ttf", size, encoding="utf-8")
    draw.text((x, y - size), string, color, font=font)
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    return img

体育动作

In [6]:
# 最新
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from XEdu.hub import Workflow as wf
import numpy as np
import time

# 全局变量
left_button_clicked = False  # 鼠标左键是否按下
stop_drawing = False  # 是否停止绘制新的红线
running_green = False
reset = False
target_x = []  # 保存红线的 x 坐标
target_y = []  # 保存红线的 y 坐标

# 鼠标事件回调函数
def mouse_callback(event, x, y, flags, param):
    global left_button_clicked, stop_drawing, running_green
    if event == cv2.EVENT_LBUTTONDOWN:  # 鼠标左键按下
        left_button_clicked = True
        stop_drawing = False  # 恢复绘制
        running_green = True
    elif event == cv2.EVENT_RBUTTONDOWN:  # 鼠标右键按下
        stop_drawing = True  # 停止绘制新的红线

# 摄像头画面设置
cap = cv2.VideoCapture()
cap.open(0, cv2.CAP_DSHOW)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)  # 设置摄像头帧率为 30 FPS
cv2.namedWindow('Camera', cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)

# 设置鼠标回调函数
cv2.setMouseCallback('Camera', mouse_callback)

text_x = 0
text_y = 0
size = (1280, 720)  # 需要转为视频的图片的尺寸

# 初始化变量
start_time = time.time()
frame_count = 0
actual_fps = 30  # 初始值

# 加载人体关键点模型
body = wf(task='pose_body26')
det = wf(task='det_body', checkpoint=r'D:/XEdu/checkpoints/mmedu_det_model/bodydetect.onnx')  # 实例化detect模型

while True:
    try:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 计算实际帧率
        if frame_count % 30 == 0:  # 每 30 帧计算一次帧率
            elapsed_time = time.time() - start_time
            actual_fps = frame_count / elapsed_time
            print(f"实际帧率: {actual_fps:.2f} FPS")

        # 动态调整视频写入帧率
        if frame_count == 1:  # 第一帧时初始化 VideoWriter
            target_video = cv2.VideoWriter("realtime1.mp4", cv2.VideoWriter_fourcc('I', '4', '2', '0'), actual_fps, size)

        background = np.zeros((size[1], size[0], 3), np.uint8)
        resized_frame = cv2.resize(frame, size)
        screen_height, screen_width, channels = resized_frame.shape

        # 人体目标检测
        bboxs = det.inference(data=resized_frame, thr=0.3)
        if len(bboxs) == 0:
            print('未发现目标')
            continue

        closest_box = find_closest_box(bboxs, screen_width, screen_height)
        closest_index = find_box_index(bboxs, closest_box)

        # 人体关键点检测
        for i in [bboxs[closest_index]]:
            keypoints, img = body.inference(data=resized_frame, img_type='cv2', bbox=i)

        keypoints = np.concatenate(keypoints).reshape(1, -1)
        if len(keypoints) == 0:  # 如果特征数据为空则不进行模型预测
            print("未提取到关键点")
            continue

        # 绘制人体关键点
        points_dict = {}
        keypoints = keypoints[0]
        for i in range(len(keypoints) // 2):
            x, y = keypoints[i * 2], keypoints[i * 2 + 1]
            x, y = int(x), int(y)  # 将坐标转换为整数
            points_dict[i] = (x, y)
            cv2.circle(resized_frame, (x, y), 3, (0, 255, 0), -1)
            cv2.putText(resized_frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

            # 如果左键按下且未停止绘制，则记录关键点
            # if i == 19:
            
            if left_button_clicked and not stop_drawing and i == 19:
                target_x.append(x)
                target_y.append(y)
            # 关键点连线
            # 定义连接规则
        connections = [
                (0, 1), (0, 2), (1, 3), (2, 4),  # 连接身体部分
                (5, 6), (5, 7), (7, 9), (6, 8), (8, 10),  # 连接左手部分
                (11, 12), (11, 13), (13, 15), (12, 14), (14, 16),  # 连接右手部分
                (5, 11), (6, 12)  # 连接身体和头部
            ]

        for connection in connections:
            start_point = (points_dict[connection[0]][0], points_dict[connection[0]][1])
            end_point = (points_dict[connection[1]][0], points_dict[connection[1]][1])
            cv2.line(resized_frame, start_point, end_point, (0, 255, 0), 2)

        # 绘制已保存的红线

        if len(target_x) > 0:  # 确保 target_x 和 target_y 不为空
            for hip_index in range(len(target_x)):
                cv2.circle(resized_frame, (target_x[hip_index], target_y[hip_index]), 5, (0, 0, 255), -1)
                if hip_index != 0:
                    cv2.line(resized_frame, (target_x[hip_index - 1], target_y[hip_index - 1]),
                                (target_x[hip_index], target_y[hip_index]), (0, 0, 255), 5)

        # 绘制标准动作参考线
        try:
            if running_green == True:
                df_crite = pd.read_csv('coach_data.csv')
                if len(df_crite) > 0 and len(target_x) > 0:  # 确保 df_crite 和 target_x 不为空
                    offset_x = abs(df_crite['x'][0] - target_x[0])
                    offset_y = abs(df_crite['y'][0] - target_y[0])
                    offset_direction_x = True
                    offset_direction_y = True
                    if df_crite['x'][0] > target_x[0]:
                        offset_direction_x = True
                    else:
                        offset_direction_x = False
                    if df_crite['y'][0] > target_y[0]:
                        offset_direction_y = True
                    else:
                        offset_direction_y = False

                    if offset_direction_x:
                        df_crite['x'] = df_crite['x'] - offset_x
                    else:
                        df_crite['x'] = df_crite['x'] + offset_x
                    if offset_direction_y:
                        df_crite['y'] = df_crite['y'] - offset_y
                    else:
                        df_crite['y'] = df_crite['y'] + offset_y
                    
                    if left_button_clicked and not stop_drawing:
                        for crite_index in range(len(target_x)):
                            if (len(df_crite) - 2 > crite_index):
                                cv2.circle(resized_frame, (df_crite['x'][crite_index], df_crite['y'][crite_index]), 3, (0, 255, 0), -1)
                                if crite_index != 0:
                                    cv2.line(resized_frame, (df_crite['x'][crite_index - 1], df_crite['y'][crite_index - 1]),
                                                (df_crite['x'][crite_index], df_crite['y'][crite_index]), (0, 255, 0), 3)
                                if crite_index == len(target_x) - 1:
                                    text_x = df_crite['x'][crite_index]
                                    text_y = df_crite['y'][crite_index]
                    if left_button_clicked and stop_drawing:
                        for crite_index in range(len(df_crite)):
                            if (len(df_crite) - 2 > crite_index):
                                cv2.circle(resized_frame, (df_crite['x'][crite_index], df_crite['y'][crite_index]), 3, (0, 255, 0), -1)
                                if crite_index != 0:
                                    cv2.line(resized_frame, (df_crite['x'][crite_index - 1], df_crite['y'][crite_index - 1]),
                                                (df_crite['x'][crite_index], df_crite['y'][crite_index]), (0, 255, 0), 3)
                                if crite_index == len(target_x) - 1:
                                    text_x = df_crite['x'][crite_index]
                                    text_y = df_crite['y'][crite_index]
                        
                        
                    # cv2.putText(resized_frame, 'Teacher', (text_x + 10, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
                    resized_frame = draw_box_string(resized_frame, text_x + 10, text_y - 10, '教师重心', 25, (0, 255, 0))
        except Exception as e:
            print(f"读取标准动作数据失败: {e}")

        # 显示画面
        resized_frame = draw_box_string(resized_frame, 10, 30, '基于人体关键点检测的体育规范动作对比系统V1.0')
        resized_frame = draw_box_string(resized_frame, 840, 30, '北京外国语大学附属上海闵行高级中学', 25)
        resized_frame = draw_box_string(resized_frame, 1090, 60, '数据科学工作坊', 25)
        cv2.imshow('Camera', resized_frame)
        target_video.write(resized_frame)

        # 根据实际帧率调整延迟时间
        delay = int(1000 / actual_fps)  # 根据实际帧率计算延迟时间（单位：毫秒）
        key = cv2.waitKey(1) & 0xFF
        if key == ord('R') or key == ord('r'):
            print("R键被按下")
            stop_drawing = False
            left_button_clicked = False
            target_x = []  # 保存红线的 x 坐标
            target_y = []  # 保存红线的 y 坐标
            resized_frame = cv2.resize(frame, size)
        elif key == ord('q'):
            print('结束')
            break                      

    except Exception as e:
        print(f"发生错误: {e}")
        break

# 释放资源
cap.release()
target_video.release()
cv2.destroyAllWindows()
print('程序结束')

模型加载成功！
模型加载成功！
实际帧率: 10.85 FPS
实际帧率: 11.77 FPS
实际帧率: 12.11 FPS
未发现目标
实际帧率: 12.28 FPS
实际帧率: 12.39 FPS
实际帧率: 12.47 FPS
实际帧率: 12.53 FPS
实际帧率: 12.57 FPS
实际帧率: 12.60 FPS
实际帧率: 12.62 FPS
实际帧率: 12.64 FPS
实际帧率: 12.66 FPS
实际帧率: 12.68 FPS
实际帧率: 12.69 FPS
实际帧率: 12.70 FPS
实际帧率: 12.71 FPS
实际帧率: 12.72 FPS
实际帧率: 12.73 FPS
实际帧率: 12.73 FPS
实际帧率: 12.74 FPS
实际帧率: 12.75 FPS
实际帧率: 12.71 FPS
实际帧率: 12.68 FPS
实际帧率: 12.64 FPS
实际帧率: 12.61 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 12.61 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 12.62 FPS
实际帧率: 12.62 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 12.61 FPS
未发现目标
未发现目标
实际帧率: 12.59 FPS
未发现目标
未发现目标
未发现目标
实际帧率: 12.57 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 12.59 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 12.58 FPS
实际帧率: 12.56 FPS
实际帧率: 12.53 FP

未发现目标
未发现目标
实际帧率: 11.35 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 11.36 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 11.37 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 11.37 FPS
实际帧率: 11.36 FPS
实际帧率: 11.36 FPS
实际帧率: 11.36 FPS
实际帧率: 11.35 FPS
实际帧率: 11.35 FPS
实际帧率: 11.35 FPS
实际帧率: 11.35 FPS
实际帧率: 11.34 FPS
实际帧率: 11.34 FPS
实际帧率: 11.34 FPS
实际帧率: 11.34 FPS
实际帧率: 11.33 FPS
实际帧率: 11.33 FPS
实际帧率: 11.33 FPS
实际帧率: 11.33 FPS
实际帧率: 11.32 FPS
实际帧率: 11.32 FPS
实际帧率: 11.32 FPS
实际帧率: 11.32 FPS
实际帧率: 11.31 FPS
实际帧率: 11.31 FPS
实际帧率: 11.31 FPS
实际帧率: 11.31 FPS
实际帧率: 11.30 FPS
实际帧率: 11.30 FPS
实际帧率: 11.30 FPS
实际帧率: 11.30 FPS
实际帧率: 11.30 FPS
实际帧率: 11.29 FPS
实际帧率: 11.29 FPS
实际帧率: 11.29 FPS
实际帧率: 11.29 FPS
实际帧率: 11.2

实际帧率: 10.70 FPS
实际帧率: 10.69 FPS
实际帧率: 10.69 FPS
实际帧率: 10.69 FPS
实际帧率: 10.69 FPS
实际帧率: 10.69 FPS
实际帧率: 10.69 FPS
实际帧率: 10.69 FPS
实际帧率: 10.68 FPS
实际帧率: 10.68 FPS
实际帧率: 10.68 FPS
实际帧率: 10.68 FPS
实际帧率: 10.68 FPS
实际帧率: 10.68 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.67 FPS
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.65 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.64 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.63 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10.62 FPS
实际帧率: 10

未发现目标
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.66 FPS
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.66 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
实际帧率: 10.67 FPS
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目标
未发现目

我在jupyter notebook中写了一段基于人体关键点检测的运动员重心轨迹可视化代码，可以将摄像头画面中的人物重心移动轨迹实时显示在opencv的窗口上。我想进一步用pyqt做一个用户界面，像个软件一样在主界面分别一左一右显示摄像头原始画面和结果画面。同时还具备切换电脑上不同的摄像头。同时还支持从本地读取视频文件，用于绘制画面。
请你在我的代码基础上修改，以下是我的代码：
# 最新
import pandas as pd
import cv2
import matplotlib.pyplot as plt
from XEdu.hub import Workflow as wf
import numpy as np
import time

# 判断居中人体框
def get_center(box):
    x1, y1, x2, y2 = box
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    return center_x, center_y

def find_closest_box(boxes, screen_width, screen_height):
    screen_center = (screen_width / 2, screen_height / 2)
    min_distance = float('inf')
    closest_box = None

    for box in boxes:
        center = get_center(box)
        distance = np.sqrt((center[0] - screen_center[0]) ** 2 + (center[1] - screen_center[1]) ** 2)
        if distance < min_distance:
            min_distance = distance
            closest_box = box

    return closest_box
def find_box_index(boxes, target_box):
    for index, box in enumerate(boxes):
        if np.all(box == target_box):
            return index
    return -1  # 如果找不到目标box，返回-1
# 全局变量
left_button_clicked = False  # 鼠标左键是否按下
stop_drawing = False  # 是否停止绘制新的红线
running_green = False
target_x = []  # 保存红线的 x 坐标
target_y = []  # 保存红线的 y 坐标

# 鼠标事件回调函数
def mouse_callback(event, x, y, flags, param):
    global left_button_clicked, stop_drawing, running_green
    if event == cv2.EVENT_LBUTTONDOWN:  # 鼠标左键按下
        left_button_clicked = True
        stop_drawing = False  # 恢复绘制
        running_green = True
    elif event == cv2.EVENT_RBUTTONDOWN:  # 鼠标右键按下
        stop_drawing = True  # 停止绘制新的红线

# 摄像头画面设置
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_FPS, 30)  # 设置摄像头帧率为 30 FPS
cv2.namedWindow('Camera', cv2.WINDOW_NORMAL | cv2.WINDOW_KEEPRATIO)

# 设置鼠标回调函数
cv2.setMouseCallback('Camera', mouse_callback)

text_x = 0
text_y = 0
size = (1280, 720)  # 需要转为视频的图片的尺寸

# 初始化变量
start_time = time.time()
frame_count = 0
actual_fps = 30  # 初始值

# 加载人体关键点模型
body = wf(task='pose_body26')
det = wf(task='det_body', checkpoint=r'D:/XEdu/checkpoints/mmedu_det_model/bodydetect.onnx')  # 实例化detect模型

while True:
    try:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # 计算实际帧率
        if frame_count % 30 == 0:  # 每 30 帧计算一次帧率
            elapsed_time = time.time() - start_time
            actual_fps = frame_count / elapsed_time
            print(f"实际帧率: {actual_fps:.2f} FPS")

        # 动态调整视频写入帧率
        if frame_count == 1:  # 第一帧时初始化 VideoWriter
            target_video = cv2.VideoWriter("realtime1.mp4", cv2.VideoWriter_fourcc('I', '4', '2', '0'), actual_fps, size)

        background = np.zeros((size[1], size[0], 3), np.uint8)
        resized_frame = cv2.resize(frame, size)
        screen_height, screen_width, channels = resized_frame.shape

        # 人体目标检测
        bboxs = det.inference(data=resized_frame, thr=0.3)
        if len(bboxs) == 0:
            print('未发现目标')
            continue

        closest_box = find_closest_box(bboxs, screen_width, screen_height)
        closest_index = find_box_index(bboxs, closest_box)

        # 人体关键点检测
        for i in [bboxs[closest_index]]:
            keypoints, img = body.inference(data=resized_frame, img_type='cv2', bbox=i)

        keypoints = np.concatenate(keypoints).reshape(1, -1)
        if len(keypoints) == 0:  # 如果特征数据为空则不进行模型预测
            print("未提取到关键点")
            continue

        # 绘制人体关键点
        points_dict = {}
        keypoints = keypoints[0]
        for i in range(len(keypoints) // 2):
            x, y = keypoints[i * 2], keypoints[i * 2 + 1]
            x, y = int(x), int(y)  # 将坐标转换为整数
            points_dict[i] = (x, y)
            cv2.circle(resized_frame, (x, y), 3, (0, 255, 0), -1)
            cv2.putText(resized_frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)

            # 如果左键按下且未停止绘制，则记录关键点
            # if i == 19:
            if left_button_clicked and not stop_drawing and i == 19:
                target_x.append(x)
                target_y.append(y)

        # 绘制已保存的红线

        if len(target_x) > 0:  # 确保 target_x 和 target_y 不为空
            for hip_index in range(len(target_x)):
                cv2.circle(resized_frame, (target_x[hip_index], target_y[hip_index]), 5, (0, 0, 255), -1)
                if hip_index != 0:
                    cv2.line(resized_frame, (target_x[hip_index - 1], target_y[hip_index - 1]),
                                (target_x[hip_index], target_y[hip_index]), (0, 0, 255), 5)

        # 绘制标准动作参考线
        try:
            if running_green == True:
                df_crite = pd.read_csv('target_data.csv')
                if len(df_crite) > 0 and len(target_x) > 0:  # 确保 df_crite 和 target_x 不为空
                    offset_x = abs(df_crite['x'][0] - target_x[0])
                    offset_y = abs(df_crite['y'][0] - target_y[0])
                    offset_direction_x = True
                    offset_direction_y = True
                    if df_crite['x'][0] > target_x[0]:
                        offset_direction_x = True
                    else:
                        offset_direction_x = False
                    if df_crite['y'][0] > target_y[0]:
                        offset_direction_y = True
                    else:
                        offset_direction_y = False

                    if offset_direction_x:
                        df_crite['x'] = df_crite['x'] - offset_x
                    else:
                        df_crite['x'] = df_crite['x'] + offset_x
                    if offset_direction_y:
                        df_crite['y'] = df_crite['y'] - offset_y
                    else:
                        df_crite['y'] = df_crite['y'] + offset_y
                    
                    if left_button_clicked and not stop_drawing:
                        for crite_index in range(len(target_x)):
                            if (len(df_crite) - 2 > crite_index):
                                cv2.circle(resized_frame, (df_crite['x'][crite_index], df_crite['y'][crite_index]), 3, (0, 255, 0), -1)
                                if crite_index != 0:
                                    cv2.line(resized_frame, (df_crite['x'][crite_index - 1], df_crite['y'][crite_index - 1]),
                                                (df_crite['x'][crite_index], df_crite['y'][crite_index]), (0, 255, 0), 3)
                                if crite_index == len(target_x) - 1:
                                    text_x = df_crite['x'][crite_index]
                                    text_y = df_crite['y'][crite_index]
                    if left_button_clicked and stop_drawing:
                        for crite_index in range(len(df_crite)):
                            if (len(df_crite) - 2 > crite_index):
                                cv2.circle(resized_frame, (df_crite['x'][crite_index], df_crite['y'][crite_index]), 3, (0, 255, 0), -1)
                                if crite_index != 0:
                                    cv2.line(resized_frame, (df_crite['x'][crite_index - 1], df_crite['y'][crite_index - 1]),
                                                (df_crite['x'][crite_index], df_crite['y'][crite_index]), (0, 255, 0), 3)
                                if crite_index == len(target_x) - 1:
                                    text_x = df_crite['x'][crite_index]
                                    text_y = df_crite['y'][crite_index]
                        
                        
                    cv2.putText(resized_frame, 'Teacher', (text_x + 10, text_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
        except Exception as e:
            print(f"读取标准动作数据失败: {e}")

        # 显示画面
        cv2.imshow('Camera', resized_frame)
        target_video.write(resized_frame)

        # 根据实际帧率调整延迟时间
        delay = int(1000 / actual_fps)  # 根据实际帧率计算延迟时间（单位：毫秒）
        if cv2.waitKey(delay) & 0xFF == ord('q'):
            print('结束')
            break

    except Exception as e:
        print(f"发生错误: {e}")
        break

# 释放资源
cap.release()
target_video.release()
cv2.destroyAllWindows()
print('程序结束')